In [37]:
import pandas as pd
import numpy as np

In [38]:
# Read a CSV file and convert it to a data frame
df = pd.read_csv('Finance_all.csv')

# Find all rows that are exactly the same
duplicated_rows = df.duplicated()

# Remove all duplicated rows
df = df.drop_duplicates()

# Delete rows with empty name columns
df = df.dropna(subset=['name'])

# Show basic statistics for each column in the data box
print(df.describe())
print(df.info())

         name salary company address experience eduBack companyType   
count   74983  74983   74983   74983      74983   74983       71838  \
unique  40154    647   13039      71          7      13          16   
top      客户经理  1万-2万  上海链家地产   北京-朝阳         不限      专科          民营   
freq     1601   5664    1097    7828      33180   26565       29788   

             scale    info  
count        74154   74983  
unique           7   45375  
top     1000-9999人  ['全职']  
freq         19303     562  
<class 'pandas.core.frame.DataFrame'>
Index: 74983 entries, 0 to 87115
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   name         74983 non-null  object
 1   salary       74983 non-null  object
 2   company      74983 non-null  object
 3   address      74983 non-null  object
 4   experience   74983 non-null  object
 5   eduBack      74983 non-null  object
 6   companyType  71838 non-null  object
 7   scale        74154 non

In [39]:
# Define a function that determines the city name and city id based on the value in the address column
def get_city_info(address):
    city_id = None
    city_name = None
    if '北京' in address:
        city_id = 530
        city_name = '北京'
    elif '上海' in address:
        city_id = 538
        city_name = '上海'
    elif '广州' in address:
        city_id = 763
        city_name = '广州'
    elif '深圳' in address:
        city_id = 765
        city_name = '深圳'
    elif '杭州' in address:
        city_id = 653
        city_name = '杭州'
    return pd.Series({'city_id': city_id, 'city_name': city_name})

# Add city_name and city_id to the data frame using the apply() function and a lambda expression
df[['city_id', 'city_name']] = df['address'].apply(lambda x: get_city_info(x))

In [40]:
df = df.dropna(subset=['city_id'])
df['city_id'] = df['city_id'].astype(int)

In [41]:
# Combine similar education levels
df['eduBack'] = df['eduBack'].replace(['EMBA', 'MBA/EMBA', '硕士'], '硕士')
df['eduBack'] = df['eduBack'].replace(['大专','专科'], '大专')
df['eduBack'] = df['eduBack'].replace(['中专/中技', '中技'], '中专')
df['eduBack'] = df['eduBack'].replace(['高中','初中及以下'], '高中及以下')

# Map education requirements to corresponding codes
edu_map = {
    '学历不限': 0,
    '高中及以下':1,
    '中专':2,
    '大专':3,
    '本科': 4,
    '硕士': 5,
    '博士': 6
}

df['eduBack_code'] = df['eduBack'].map(edu_map)

In [42]:
edu_en = {
    '学历不限': 'No Education Required',
    '高中及以下':'High School and below',
    '中专': 'Technical Secondary',
    '大专': 'Junior College',
    '本科': 'Bachelor',
    '硕士': 'Master',
    '博士': 'Doctor'
}
df['eduBack_en'] = df['eduBack'].map(edu_en)
city_en = {530: 'Beijing', 538: 'Shanghai', 763: 'Guangzhou', 765: 'Shenzhen', 653: 'Hangzhou'}
df['city_en'] = df['city_id'].map(city_en)

In [43]:
# Calculate value counts and percentages
edu_count = df['eduBack_en'].value_counts()
edu_percentages = round((edu_count / edu_count.sum()) * 100,2)

edu_summary = pd.DataFrame({'Count': edu_count, 'Percentage': edu_percentages})
edu_summary.reset_index(inplace=True)
edu_summary.columns = ['eduBack_en', 'Count', 'Percentage']

print(edu_summary)

              eduBack_en  Count  Percentage
0         Junior College  31106       41.48
1               Bachelor  26441       35.26
2  No Education Required   9140       12.19
3  High School and below   4127        5.50
4                 Master   3574        4.77
5    Technical Secondary    517        0.69
6                 Doctor     78        0.10


In [44]:
df['scale'] = df['scale'].str.replace('人', '')
df['scale'] = df['scale'].str.replace('以上', '+')
df['scale'] = df['scale'].str.replace('以下', '-')
# Fill empty or blank values with a default code of 7
df['scale'] = df['scale'].fillna('other')

# Map education requirements to corresponding codes
scale_map = {
    '20-': 0,
    '20-99': 1,
    '100-299': 2,
    '300-499': 3,
    '500-999': 4,
    '1000-9999': 5,
    '10000+': 6,
    'other': 7
}

df['scale_code'] = df['scale'].map(scale_map)

In [45]:
scale_id = {
    0:'Less than 20',
    1:'20-99',
    2:'100-299',
    3:'300-499',
    4:'500-999',
    5:'1000-9999',
    6:'Over 10000',
    7: 'Other'
}
df['scale_en'] = df['scale_code'].map(scale_id)

In [46]:
# Calculate value counts and percentages
sca_count = df['scale'].value_counts()
sca_percentages = round((sca_count / sca_count.sum()) * 100,2)

sca_summary = pd.DataFrame({'Count': sca_count, 'Percentage': sca_percentages})
sca_summary.reset_index(inplace=True)
sca_summary.columns = ['scale', 'Count', 'Percentage']

print(sca_summary)

       scale  Count  Percentage
0  1000-9999  19303       25.74
1     10000+  17997       24.00
2    100-299  12404       16.54
3      20-99  11951       15.94
4    500-999   7242        9.66
5    300-499   3093        4.12
6        20-   2164        2.89
7      other    829        1.11


In [47]:
df['experience'] = df['experience'].str.replace('年', 'year')
df['experience'] = df['experience'].str.replace('以上', '+')
df['experience'] = df['experience'].str.replace('以下', '-')
df['experience'] = df['experience'].replace(['无经验','不限'], 'No Work Experience Required')

exp_map = {
    '1year-': 0,
    '1-3year': 1,
    '3-5year': 2,
    '5-10year': 3,
    '10year+': 4,
    'No Work Experience Required': 5
}

df['exp_code'] = df['experience'].map(exp_map)

In [48]:
exp_id = {
    0:'Less than 1 year',
    1:'1-3 years',
    2:'3-5 years',
    3:'5-10 years',
    4:'Over 10 years',
    5:'No Work Experience Required'
}
df['exp_en'] = df['exp_code'].map(exp_id)

In [49]:
# Calculate value counts and percentages
exp_count = df['exp_en'].value_counts()
exp_percentages = round((exp_count / exp_count.sum()) * 100,2)

exp_summary = pd.DataFrame({'Count': exp_count, 'Percentage': exp_percentages})
exp_summary.reset_index(inplace=True)
exp_summary.columns = ['exp_en', 'Count', 'Percentage']

print(exp_summary)

                        exp_en  Count  Percentage
0  No Work Experience Required  38375       51.18
1                    1-3 years  20809       27.75
2                    3-5 years   9609       12.81
3                   5-10 years   4179        5.57
4             Less than 1 year   1564        2.09
5                Over 10 years    447        0.60


In [50]:
df = df[~df['salary'].isin(['面议']) & 
        ~df['salary'].str.contains('天') &
        ~df['salary'].str.contains('时') &
        ~df['salary'].str.contains('次') &
        ~df['salary'].str.contains('下')]

In [51]:
def bottom_top(x,num=0):
    """Clean up each row of payroll data to unify the payroll 
    in thousands. Split into two columns and return the specified 
    column with num=0 for the lowest column and num=1 
    for the highest column."""
    if x == '面议' or x.find('天') != -1 or x.find('时') != -1 or x.find('次') != -1 or x.find('下') != -1:
        return np.nan
    elif x.find('/月') != -1:
        x = x.replace('/月', '')
    elif x.endswith('元/月'):
        x = float(x[:-3])/1000
    if len(x.split('-')) >1:
        x = x.split('-')[num]
        if x.find('千') != -1:
            x=float(x[0:x.find('千')])
        elif x.find('万') != -1:
            x=float(x[0:x.find('万')])*10
        elif x.find('') != -1:
            x=0
        return x


df=df.drop(index = df[df['salary'] == 0].index )
df['bottom'] = df.salary.apply(lambda x : bottom_top(x,0))
df['top'] = df.salary.apply(lambda x : bottom_top(x,1))
df['avg'] = (df['bottom']+df['top'])/2
df['avg_w'] =df['avg']/10

In [52]:
#Add a new column salary_label to classify the average salary into six categories: <5k,5k-1w,1w-1w5,1w5-2w,2w-5w,>5w.
df['salary_label'] = pd.cut(df.avg, bins=[0, 5, 10, 15,20, 50, max(df.avg)], labels=['<5000', '5000-10000', '10000-15000','15000-20000', '20000-50000', '>50000'])

In [53]:
# Calculate value counts and percentages
sal_count = df['salary_label'].value_counts()
sal_percentages = round((sal_count / sal_count.sum()) * 100,2)

sal_summary = pd.DataFrame({'Count': sal_count, 'Percentage': sal_percentages})
sal_summary.reset_index(inplace=True)
sal_summary.columns = ['sal_en', 'Count', 'Percentage']

print(sal_summary)

        sal_en  Count  Percentage
0  10000-15000  24385       34.31
1   5000-10000  19861       27.95
2  20000-50000  12741       17.93
3  15000-20000  11416       16.06
4        <5000   2025        2.85
5       >50000    638        0.90


In [54]:
unique_company_types = df['companyType'].unique()
co_map={'民营':'Private', '上市公司':'Public Company', '股份制企业':'Joint-Stock enterprise', '国企':'State-Owned Enterprise', '合资':'Joint Venture', '外商独资':'Wholly Foreign-Owned', '其他':'Other', '事业单位':'Public Institution','银行':'Bank'}
df['Comp_en']=df['companyType'].map(co_map)

In [55]:
keywords = ['投资经理','风控','催收','银行柜员',
                '银行销售','信审','信用卡','贷款',
                '金融产品','汽车金融','金融研究',
                '证券','交易员','期货',
                '操盘手','基金','股票','投资顾问',
                '信托','典当','担保','信贷','权证',
                '财产保险','保险内勤','理赔','精算师',
                '保险销售','理财顾问','查勘定损','车险']

df['keywords'] = df['name'].apply(lambda x: [keyword for keyword in keywords if keyword.lower() in x.lower()])


In [56]:
def classify_industry(row):
    keywords = row['keywords']
    if any(keyword in keywords for keyword in ["投资经理", "金融研究", "证券", "交易员", "期货", "操盘手", "基金", "股票", "投资顾问","金融产品","信托","权证"]):
        return "Investment"
    elif any(keyword in keywords for keyword in ["银行柜员", "银行销售", "贷款","风控","信审", "信用卡", "担保", "信贷","查勘定损","催收"]):
        return "Banking"
    elif any(keyword in keywords for keyword in ["财产保险", "保险内勤", "理赔", "精算师", "保险销售", "理财顾问",'查勘定损','车险']):
        return "Insurance"
    else:
        return "Other"


df['industry_category'] = df.apply(classify_industry, axis=1)

In [57]:
# Calculate value counts and percentages
cat_count = df['industry_category'].value_counts()
cat_percentages = round((cat_count / cat_count.sum()) * 100,2)

cat_summary = pd.DataFrame({'Count': cat_count, 'Percentage': cat_percentages})
cat_summary.reset_index(inplace=True)
cat_summary.columns = ['cat_en', 'Count', 'Percentage']

print(cat_summary)

       cat_en  Count  Percentage
0       Other  55527       78.12
1  Investment   6468        9.10
2     Banking   5421        7.63
3   Insurance   3662        5.15


In [58]:
# Save the cleaned data frame as a new CSV file
df.to_csv('Finance_cleaned_file.csv', index=False)